### Początkowa konfiguracja

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from typing import Dict, List

plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (7, 5)
# plt.style.use('dark_background')


### Ładowanie danych

In [ ]:
with open("data.json") as file:
    data = json.load(file)


cmTom = 0.01  # centymetry na metry
mmTom = 0.001  # milimetry na metry

N = 300  # liczba naciec na siatce
d = 1 / N * mmTom  # stala siatki dyfrakcyjnej w [m]


# data: dict[str, list[dict[str, float]]] = {}
# data["L=0.5"] -> [{"srednia": val1, "lambda": val2}, {"srednia": val1}, {}]
# index listy to numer przypadku


### Obliczenie średniej i długości fali

In [ ]:
measurements_list = list()
lambda_list = list()

for L_key, measurement_list in data.items():

    # lista_pomiarow = lista slownikow gdzie klucz zmienna np. srednia, a value to wartość
    # index listy_pomiarow to numer przypadku

    L_value = float(L_key[2:5])  # L_value w [m]

    for measurement in measurement_list:
        average = (measurement["xL"] * cmTom + measurement["xP"] * cmTom) / 2
        temp_dict = {
            "L": L_value,
            "n": measurement["n"],
            "xL": measurement["xL"] * cmTom,  # xL w [m]
            "xP": measurement["xP"] * cmTom,  # xP w [m]
            "x_avg": average,  # srednia w [m]
            # lambda w [m]
            "lambda": d
            / measurement["n"]
            * average
            / (sqrt(average**2 + L_value**2)),
        }

        measurements_list.append(temp_dict)
        lambda_list.append(temp_dict["lambda"])

# for measurement in measurements_list:
#    print(measurement)


### Obliczenie niepewności $\lambda$

In [ ]:
uL = 0.5 * cmTom  # niepewnosc L w [m]
uxL = 0.3 * cmTom  # niepewnosc xL w [m]
uxP = 0.3 * cmTom  # niepewnosc xP w [m]

uxSr = 0.00212  # niepewnosc xSr w [m]

# \ = d/n * xsr / sqrt(xsr^2 + L^2)
# d i n to stałe, xsr i L zmienne

N = 300  # liczba naciec na siatce
d = 1 / N * mmTom  # stala siatki dyfrakcyjnej w [m]


def niepewnosc_lambda(measurements_list: List[Dict[str, float]]) -> List[float]:
    errors = list()
    for measurement in measurements_list:
        L = measurement["L"]
        n = measurement["n"]
        xn = measurement["x_avg"]

        pochodna_po_xn = (uxSr * d * L**2) / (n * (xn**2 + L**2) ** 1.5)
        pochodna_po_l = (uL * -d * L * xn) / (n * ((xn**2 + L**2) ** 1.5))
        errors.append(sqrt(pochodna_po_xn**2 + pochodna_po_l**2))

    return errors


lambda_errors = niepewnosc_lambda(measurements_list)
print(lambda_errors)


### Obliczenie średniej ważonej z lambdy i jej niepewności

In [ ]:
def srednia_wazona(lambda_errors: List[float], lambda_list: List[float]) -> float:
    wages = [1 / (u * u) for u in lambda_errors]
    return sum(wage * val for wage, val in zip(wages, lambda_list)) / sum(wages)


def niepewnosc_sredniej_wazonej(lambda_errors):
    wages = [1 / (u * u) for u in lambda_errors]
    return 1 / sqrt(sum(wages))


lambda_srednia_wazona = srednia_wazona(lambda_errors, lambda_list)
lambda_srednia_wazona_error = niepewnosc_sredniej_wazonej(lambda_errors)

print(lambda_srednia_wazona)
print(lambda_srednia_wazona_error)


### Zapisanie długości fali w nano metrach

In [ ]:
mTonm = 10**9  # metry na nanometry

print("\nwartosci lambda w nanometrach")
lambda_list = [mTonm * l for l in lambda_list]
print(lambda_list)

print("\nniepewnosc lambda w nanometrach:")
lambda_errors = [mTonm * l for l in lambda_errors]
print(lambda_errors)

print("\nsrednia wazona z lambdy i jej niepewnosci w nanometrach:")
lambda_srednia_wazona *= mTonm
print(lambda_srednia_wazona)

print("\nniepewnosc sredniej wazonej z lambda w nanometrach:")
lambda_srednia_wazona_error *= mTonm
print(lambda_srednia_wazona_error)
